# Projeto de Análise de Dados

## Importando as bibliotecas e o Arquivo

In [ ]:
pip install inflection

In [ ]:
# Bibliotecas necessárias

import pandas as pd
import numpy as np
# import streamlit as st
import plotly.express as px
import inflection
import plotly.graph_objs as go
from google import colab as cl

# importando o arquivo para o computador
file_upload = cl.files.upload()

Saving dados_processados.csv to dados_processados.csv


In [ ]:
# 1. Lendo o arquivo zomato.csv
file_path = 'dados_processados.csv'
data = pd.read_csv(file_path)

## Carregando os dados e definindo funções auxiliares

In [ ]:
#Criando um dicionário com os códigos de países

COUNTRIES = {
    1: "India",
    14: "Australia",
    30: "Brazil",
    37: "Canada",
    94: "Indonesia",
    148: "New Zeland",
    162: "Philippines",
    166: "Qatar",
    184: "Singapure",
    189: "South Africa",
    191: "Sri Lanka",
    208: "Turkey",
    214: "United Arab Emirates",
    215: "England",
    216: "United States of America",
}

In [ ]:
#Criando um dicionário com os códigos de cores

COLORS = {
    "3F7E00": "darkgreen",
    "5BA829": "green",
    "9ACD32": "lightgreen",
    "CDD614": "orange",
    "FFBA00": "red",
    "CBCBC8": "darkred",
    "FF7800": "darkred",
}

In [ ]:
#Função para mostrar as dimensões do dataframe
def show_dataframe_dimensions(dataframe):
    print(f"Number of Rows: {dataframe.shape[0]}")
    print(f"Number of Columns: {dataframe.shape[1]}")

    return None

show_dataframe_dimensions(dados)

Number of Rows: 7527
Number of Columns: 21


In [ ]:
#Funçao para identificar os atributos(variaveis) numericos

def get_numerical_attributes(dataframe):
    return dataframe.select_dtypes(include=['int64', 'float64'])

get_numerical_attributes(dados)

,Restaurant ID,Country Code,Longitude,Latitude,Average Cost for two,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Votes
0,6310675,162,121.009787,14.447615,1100,1,0,0,0,3,4.6,619
1,6310675,162,121.009787,14.447615,1100,1,0,0,0,3,4.6,619
2,6314542,162,121.024562,14.556042,3100,0,0,0,0,4,4.7,469
3,6301293,162,121.023171,14.556196,800,0,0,0,0,3,4.4,867
4,6315689,162,121.027708,14.565899,700,0,0,0,0,3,4.4,858
...,...,...,...,...,...,...,...,...,...,...,...,...
7522,5912546,208,29.017326,41.065322,300,0,0,0,0,4,4.3,1367
7523,5913006,208,29.052620,41.050280,45,0,0,0,0,2,4.5,1172
7524,5923535,208,29.052623,41.050717,140,0,0,0,0,4,4.2,1160
7525,5914190,208,29.016590,41.025741,95,0,0,0,0,3,4.0,770


In [ ]:
#Funcao para calcular estatísticas básicas a partir dos dados numericos

def get_first_order_statistics(dataframe):
    # Central Tendency Metrics
    mean = pd.DataFrame(dataframe.apply(np.mean)).T
    median = pd.DataFrame(dataframe.apply(np.median)).T

    # Dispersion Metrics
    min_ = pd.DataFrame(dataframe.apply(min)).T
    max_ = pd.DataFrame(dataframe.apply(max)).T
    range_ = pd.DataFrame(dataframe.apply(lambda x: x.max() - x.min())).T
    std = pd.DataFrame(dataframe.apply(np.std)).T
    skew = pd.DataFrame(dataframe.apply(lambda x: x.skew())).T
    kurtosis = pd.DataFrame(dataframe.apply(lambda x: x.kurtosis())).T

    # Metrics Concatenation
    m = pd.concat([min_, max_, range_, mean, median, std, skew, kurtosis]).T.reset_index()
    m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']

    return m

In [ ]:
#Funçao para renomear as variaveis

def rename_columns(dataframe):
    df = dataframe.copy()
    title = lambda x: inflection.titleize(x)
    snakecase = lambda x: inflection.underscore(x)
    spaces = lambda x: x.replace(" ", "")
    cols_old = list(df.columns)
    cols_old = list(map(title, cols_old))
    cols_old = list(map(spaces, cols_old))
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new
    return df

In [ ]:
# Funçao para substituir os códigos dos países por seus nomes de acordo com o dicionário COUNTRIES
def country_name(country_id):
    return COUNTRIES[country_id]

In [ ]:
#Funçao para substituir os códigos das cores por seus nomes de acordo com o dicionário COLORS
def color_name(color_code):
    return COLORS[color_code]

In [ ]:
#Funçao para criar categorias de preços

def create_price_tye(price_range):
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
        return "normal"
    elif price_range == 3:
        return "expensive"
    else:
        return "gourmet"

In [ ]:
#Funçao para reordenar as colunas

def adjust_columns_order(dataframe):
    df = dataframe.copy()
    new_cols_order = [
        "restaurant_id",
        "restaurant_name",
        "country",
        "city",
        "address",
        "locality",
        "locality_verbose",
        "longitude",
        "latitude",
        "cuisines",
        "price_type",
        "price_range",
        "average_cost_for_two",
        "currency",
        "has_table_booking",
        "has_online_delivery",
        "is_delivering_now",
        "aggregate_rating",
        "rating_color",
        "color_name",
        "rating_text",
        "votes",
    ]
    return df.loc[:, new_cols_order]

In [ ]:
#Funçao para aplicar todas as funçoes criadas e  tratar os dados de uma única vez

def process_data(file_path):
    df = pd.read_csv(file_path) #Lendo o CSV

    df = df.dropna() #Removendo as linhas com NAs

    df = rename_columns(df) #Renomeando as colunas

    df["price_type"] = df.loc[:, "price_range"].apply(lambda x: create_price_tye(x)) #Criando as categorias de preços

    df["country"] = df.loc[:, "country_code"].apply(lambda x: country_name(x)) #Trocando códigos por nomes de países

    df["color_name"] = df.loc[:, "rating_color"].apply(lambda x: color_name(x)) #Trocando códigos por nomes de cores

    df["cuisines"] = df.loc[:, "cuisines"].astype(str).apply(lambda x: x.split(",")[0]) #Dividindo a coluna de 'cuisines'

    df = df.drop_duplicates() #Removendo as duplicatas

    df = adjust_columns_order(df) #Arrumando a ordem das colunas

    df.to_csv("dados_processados.csv", index=False) #Salvando de volta os dados processdos

    return df

## Extração e limpeza dos dados

1. Renomear as colunas
2. Fazer uma cópia de segurança dos dados
3. Identificar e remover os valores faltantes (NA)
4. Remover as linhas duplicadas
5. Converter as colunas com códigos (Country_Code, Rating_color)
6. Abrir a coluna Cuisines

In [ ]:
# Criando uma variável chamada 'data' e atribuindo a ela a última função criada chamada 'process_data' que tratou o nosso arquivo
data = process_data(file_path)

In [ ]:
# Comando para visualizar as primeiras cinco linhas do nosso DF tratado chamado 'data'
data.head()

,restaurant_id,restaurant_name,country,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,average_cost_for_two,currency,has_table_booking,has_online_delivery,is_delivering_now,aggregate_rating,rating_color,color_name,rating_text,votes
0,6310675,Mama Lou's Italian Kitchen,Philippines,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,1100,Botswana Pula(P),1,0,0,4.6,3F7E00,darkgreen,Excellent,619
1,6314542,Blackbird,Philippines,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,European,...,3100,Botswana Pula(P),0,0,0,4.7,3F7E00,darkgreen,Excellent,469
2,6301293,Banapple,Philippines,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,Filipino,...,800,Botswana Pula(P),0,0,0,4.4,5BA829,green,Very Good,867
3,6315689,Bad Bird,Philippines,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,...,700,Botswana Pula(P),0,0,0,4.4,5BA829,green,Very Good,858
4,6304833,Manam,Philippines,Makati City,"Level 1, Greenbelt 2, Ayala Center, Greenbelt,...","Greenbelt 2, San Lorenzo, Makati City","Greenbelt 2, San Lorenzo, Makati City, Makati ...",121.020380,14.552351,Filipino,...,700,Botswana Pula(P),0,0,0,4.7,3F7E00,darkgreen,Excellent,930


In [ ]:
# 1. Comando para verificar os tipos de dados de cada coluna
data.dtypes

restaurant_id             int64
restaurant_name          object
country                  object
city                     object
address                  object
locality                 object
locality_verbose         object
longitude               float64
latitude                float64
cuisines                 object
price_type               object
price_range               int64
average_cost_for_two      int64
currency                 object
has_table_booking         int64
has_online_delivery       int64
is_delivering_now         int64
aggregate_rating        float64
rating_color             object
color_name               object
rating_text              object
votes                     int64
dtype: object

In [ ]:
#Verificando estatísticas básicas dos dados

get_first_order_statistics(get_numerical_attributes(data))

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,restaurant_id,549.000000,1.904028e+07,1.903973e+07,1.074958e+07,7.900139e+06,7.037873e+06,-0.138598,-1.714864
1,longitude,-122.700455,1.753106e+02,2.980110e+02,3.145010e+01,7.375779e+01,7.902858e+01,-0.613523,-0.733448
2,latitude,-41.330428,5.597698e+01,9.730741e+01,1.941570e+01,2.526666e+01,2.351556e+01,-1.073323,0.542466
3,price_range,1.000000,4.000000e+00,3.000000e+00,2.633280e+00,3.000000e+00,9.656509e-01,-0.056585,-0.998100
4,average_cost_for_two,0.000000,2.500002e+07,2.500002e+07,7.520077e+03,2.500000e+02,3.025129e+05,81.424161,6725.361462
5,has_table_booking,0.000000,1.000000e+00,1.000000e+00,6.061481e-02,0.000000e+00,2.386224e-01,3.683479,11.571359
6,has_online_delivery,0.000000,1.000000e+00,1.000000e+00,3.504113e-01,0.000000e+00,4.770988e-01,0.627212,-1.607069
7,is_delivering_now,0.000000,1.000000e+00,1.000000e+00,1.749170e-01,0.000000e+00,3.798961e-01,1.711802,0.930533
8,aggregate_rating,0.000000,4.900000e+00,4.900000e+00,4.131116e+00,4.200000e+00,6.622396e-01,-3.950632,21.659622
9,votes,0.000000,4.133300e+04,4.133300e+04,6.053591e+02,3.280000e+02,1.116459e+03,11.377346,291.232704


In [ ]:
data.shape

(6929, 22)

# Geral

## 1. Quantos restaurantes únicos estão registrados?


In [ ]:
data.restaurant_id.nunique()

6929

## 2. Quantos países únicos estão registrados?


In [ ]:
data.country.nunique()

15

## 3. Quantas cidades únicas estão registradas?






In [ ]:
data.city.nunique()

125

## 4. Qual o total de avaliações feitas?

In [ ]:
data.votes.sum()

4194533

## 5. Qual o total de tipos de culinária registrados?

In [ ]:
cuisines = data.cuisines.nunique()
cuisines

165

# País

## 1. Qual o nome do país que possui mais cidades registradas?


In [ ]:
(data[['country', 'city']]
 .groupby('country')
 .nunique('city')
 .sort_values(by = 'city', ascending=False)
 .reset_index()).head(1)

,country,city
0,India,49


## 2. Qual o nome do país que possui mais restaurantes registrados?


In [ ]:
(data[['country', 'restaurant_id']]
.groupby('country')
.nunique()
.sort_values(by = 'restaurant_id', ascending=False)
.reset_index()).head(1)

,country,restaurant_id
0,India,3111


## 3.  Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4 registrados?







In [ ]:
(data[data['price_range'] == 4][['country', 'restaurant_id']]
 .groupby('country')
 .count()
 .sort_values(by = 'restaurant_id', ascending=False).
 reset_index()).head(1)

,country,restaurant_id
0,United States of America,415


## 4. Qual o nome do país que possui a maior quantidade de tipos de culinária distintos?

In [ ]:
(data[['country', 'cuisines']]
 .groupby('country')
 .nunique()
 .sort_values(by='cuisines', ascending=False)
 .reset_index()).head(1)

,country,cuisines
0,India,77


## 5. Qual o nome do país que possui a maior quantidade de avaliações feitas?

In [ ]:
(data[['country', 'votes']]
.groupby('country')
.sum()
.sort_values(by='votes', ascending=False)
.reset_index()).head(1)

,country,votes
0,India,2800164


## 6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem entrega?

In [ ]:
(data[data['has_online_delivery'] == 1][['country', 'restaurant_id']]
 .groupby('country')
 .count()
 .sort_values(by='restaurant_id', ascending=False)
 .reset_index()).head(1)

,country,restaurant_id
0,India,2177


## 7. Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas?


In [ ]:
(data[data['has_table_booking'] == 1][['country', 'restaurant_id']]
.groupby('country')
.nunique('restaurant_id')
.sort_values(by='restaurant_id', ascending=False)
.reset_index()).head(1)

,country,restaurant_id
0,India,256


## 8. Qual o nome do país que possui, na média, a maior quantidade de avaliações registrada?

In [ ]:
(data[['country', 'votes']]
.groupby('country')
.mean()
.sort_values(by='votes', ascending=False)
.reset_index()).head(1)

,country,votes
0,Indonesia,1112.825


## 9. Qual o nome do país que possui, na média, a maior nota média registrada?

In [ ]:
(data[['country', 'aggregate_rating']]
.groupby('country')
.mean()
.sort_values(by='aggregate_rating', ascending=False)
.reset_index()).head(1)

,country,aggregate_rating
0,Indonesia,4.60125


## 10. Qual o nome do país que possui, na média, a menor nota média registrada?

In [ ]:
(data[['country', 'aggregate_rating']]
.groupby('country')
.mean()
.sort_values(by='aggregate_rating', ascending=True)
.reset_index()).head(1)

,country,aggregate_rating
0,Brazil,3.321667


## 11. Qual a média de preço de um prato para dois por país?

In [ ]:
(data[['country', 'average_cost_for_two', 'currency']]
.groupby(['country', 'currency'])
.mean()
.round(2)
.reset_index())

,country,currency,average_cost_for_two
0,Australia,Dollar($),138959.78
1,Brazil,Brazilian Real(R$),138.81
2,Canada,Dollar($),41.86
3,England,Pounds(£),43.51
4,India,Indian Rupees(Rs.),704.40
5,Indonesia,Indonesian Rupiah(IDR),303000.00
6,New Zeland,NewZealand($),62.15
7,Philippines,Botswana Pula(P),1227.82
8,Qatar,Qatari Rial(QR),174.00
9,Singapure,Dollar($),141.44


# Cidade

## 1. Qual o nome da cidade que possui mais restaurantes registrados?


In [ ]:
(data[['city', 'restaurant_id']]
.groupby('city')
.nunique()
.sort_values(by='restaurant_id', ascending=False)
.reset_index()).head(1)

,city,restaurant_id
0,Abu Dhabi,80


## 2. Qual o nome da cidade que possui mais restaurantes com nota média acima de 4?


In [ ]:
(data[data['aggregate_rating'] >= 4][['city', 'restaurant_id']]
.groupby('city')
.count()
.sort_values(by='restaurant_id', ascending=False)
.reset_index()).head(1)

,city,restaurant_id
0,London,80


## 3.  Qual o nome da cidade que possui mais restaurantes com nota média abaixo de 2.5?







In [ ]:
(data[data['aggregate_rating'] <= 2.5][['city', 'restaurant_id']]
.groupby('city')
.count()
.sort_values(by='restaurant_id', ascending=False)
.reset_index()).head(1)

,city,restaurant_id
0,Gangtok,33


## 4. Qual o nome da cidade que possui o maior valor médio de um prato para dois?

In [ ]:
(data[['city', 'average_cost_for_two']]
.groupby('city')
.mean()
.sort_values(by='average_cost_for_two', ascending=False)
.reset_index()).head(1)

,city,average_cost_for_two
0,Adelaide,416734.133333


## 5. Qual o nome da cidade que possui a maior quantidade de tipos de culinária distintas?

In [ ]:
(data[['city', 'cuisines']]
.groupby('city')
.nunique()
.sort_values(by='cuisines', ascending=False)
.reset_index()).head(1)

,city,cuisines
0,Birmingham,32


## 6. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem reservas?

In [ ]:
(data[data['has_table_booking'] == 1][['city', 'has_table_booking']]
.groupby('city')
.sum()
.sort_values(by='has_table_booking', ascending=False).reset_index()).head(1)

,city,has_table_booking
0,Bangalore,42


## 7. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem entregas?


In [ ]:
(data[data['is_delivering_now'] == 1][['city', 'is_delivering_now']]
.groupby('city')
.sum()
.sort_values(by='is_delivering_now', ascending=False).reset_index()).head(1)

,city,is_delivering_now
0,Vadodara,48


## 8. Qual o nome da cidade que possui a maior quantidade de restaurantes que aceitam pedidos online?

In [ ]:
(data[data['has_online_delivery'] == 1][['city', 'has_online_delivery']]
.groupby('city')
.sum()
.sort_values(by='has_online_delivery', ascending=False).reset_index()).head(1)

,city,has_online_delivery
0,Bhopal,75


In [ ]:
(df1[['country', 'Votes']]
.groupby('country')
.mean()
.sort_values(by='Votes', ascending=False)
.reset_index()).head(1)

,country,Votes
0,Indonesia,1112.825


# Restaurantes

## 1. Qual o nome do restaurante que possui a maior quantidade de avaliações?





In [ ]:
cols = [
    'restaurant_id', 'restaurant_name', 'country',
    'city', 'cuisines', 'average_cost_for_two',
    'aggregate_rating', 'votes'
]

In [ ]:
(data[cols]
.sort_values(by='votes', ascending=False)
.reset_index(drop=False)).head(1)

,index,restaurant_id,restaurant_name,country,city,cuisines,average_cost_for_two,aggregate_rating,votes
0,3928,90896,Bawarchi,India,Hyderabad,Biryani,750,4.5,41333


## 2. Qual o nome do restaurante com a maior nota média?


In [ ]:
(data[cols]
.sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[False, True]) #É possível passar uma lista para o método ascending
.reset_index()).head(1)

,index,restaurant_id,restaurant_name,country,city,cuisines,average_cost_for_two,aggregate_rating,votes
0,3766,7528,Indian Grill Room,India,Gurgaon,North Indian,1800,4.9,3062


## 3. Qual o nome do restaurante que possui o maior valor de um prato para duas pessoas?








In [ ]:
(data[['restaurant_name', 'restaurant_id', 'average_cost_for_two']]
.sort_values(by=['average_cost_for_two', 'restaurant_id'], ascending=[False, True])
.reset_index()).head(1)

,index,restaurant_name,restaurant_id,average_cost_for_two
0,385,d'Arry's Verandah Restaurant,16608070,25000017


## 4. Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação?

In [ ]:
(data[data['cuisines'] == 'Brazilian'][['restaurant_name', 'restaurant_id', 'aggregate_rating']]
.sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[True, True])
.reset_index()).head(1)

,index,restaurant_name,restaurant_id,aggregate_rating
0,126,Loca Como tu Madre,6600100,0.0


## 5. Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação?

In [ ]:
(data[  (data['cuisines'] == 'Brazilian') & (data['country'] == 'Brazil') ][['restaurant_name', 'restaurant_id', 'aggregate_rating']]
.sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[False, True])
.reset_index()).head(1)

,index,restaurant_name,restaurant_id,aggregate_rating
0,207,Braseiro da Gávea,7300955,4.9


In [ ]:
(df1[  (df1['Cuisines'] == 'Brazilian') & (df1['country'] == 'Brazil') ][['Restaurant Name', 'Restaurant ID', 'Aggregate rating']]
.sort_values(by=['Aggregate rating', 'Restaurant ID'], ascending=[False, True])
.reset_index()).head(1)

,index,Restaurant Name,Restaurant ID,Aggregate rating
0,207,Braseiro da Gávea,7300955,4.9


## 6. Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?

In [ ]:
(data[['has_online_delivery', 'votes']]
.groupby('has_online_delivery')
.mean()
.round(2)
.sort_values('votes', ascending=False)
.reset_index())

,has_online_delivery,votes
0,1,838.82
1,0,479.42


## 7. Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas?


In [ ]:
(data[['has_table_booking', 'average_cost_for_two']]
.groupby('has_table_booking')
.mean()
.round(2)
.sort_values('average_cost_for_two', ascending=False)
.reset_index())

,has_table_booking,average_cost_for_two
0,1,69998.42
1,0,3488.60


## 8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ)?

In [ ]:
japa_EUA = data[(data['cuisines'] == 'Japanese') & (data['country'] == 'United States of America')]['average_cost_for_two'].mean().round(2)
BBQ_EUA = data[(data['cuisines'] == 'BBQ') & (data['country'] == 'United States of America')]['average_cost_for_two'].mean().round(2)

print(f'O valor médio de pratos de comida japonesa para duas pessoas nos EUA é de US${japa_EUA}, enquanto o de BBQ é de US${BBQ_EUA}.')

O valor médio de pratos de comida japonesa para duas pessoas nos EUA é de US$56.41, enquanto o de BBQ é de US$39.64.


# Tipos de Culinária


## 1. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a maior média de avaliação?





In [ ]:
(data[data.cuisines == 'Italian'][['restaurant_name','restaurant_id', 'aggregate_rating']]
 .sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[False, True])
 .reset_index()).head(1)

,index,restaurant_name,restaurant_id,aggregate_rating
0,5408,Darshan,10021,4.9


## 2. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a menor média de avaliação?


In [ ]:
(data[data['cuisines'] == 'Italian'][['restaurant_name', 'restaurant_id', 'aggregate_rating']]
 .sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[True, True])).head(1)

,restaurant_name,restaurant_id,aggregate_rating
150,Avenida Paulista,6600542,0.0


## 3. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a maior média de avaliação?









In [ ]:
(data[data['cuisines'] == 'American'][['restaurant_name', 'restaurant_id', 'aggregate_rating']]
 .sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[False, True])).head(1)

,restaurant_name,restaurant_id,aggregate_rating
6648,Burger & Lobster,6102616,4.9


## 4.  Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a menor média de avaliação?

In [ ]:
(data[data['cuisines'] == 'American'][['restaurant_name', 'restaurant_id', 'aggregate_rating']]
 .sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[True, True])).head(1)

,restaurant_name,restaurant_id,aggregate_rating
6751,Alston Bar & Beef,18756337,0.0


## 5. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a maior média de avaliação?

In [ ]:
(data[data['cuisines'] == 'Arabian'][['restaurant_name', 'restaurant_id', 'aggregate_rating']]
 .sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[False, True])).head(1)

,restaurant_name,restaurant_id,aggregate_rating
3896,Mandi@36,18535007,4.7


## 6. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a menor média de avaliação?

In [ ]:
(data[data['cuisines'] == 'Arabian'][['restaurant_name', 'restaurant_id', 'aggregate_rating']]
 .sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[True, True])).head(1)

,restaurant_name,restaurant_id,aggregate_rating
337,Raful,6706513,0.0


## 7. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a maior média de avaliação?


In [ ]:
(data[data['cuisines'] == 'Japanese'][['restaurant_name', 'restaurant_id', 'aggregate_rating']]
 .sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[False, True])).head(1)

,restaurant_name,restaurant_id,aggregate_rating
6633,Sushi Samba,6107336,4.9


## 8.  Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a menor média de avaliação?

In [ ]:
(data[data['cuisines'] == 'Japanese'][['restaurant_name', 'restaurant_id', 'aggregate_rating']]
 .sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[True, True])).head(1)

,restaurant_name,restaurant_id,aggregate_rating
123,Banzai Sushi,6600203,0.0


## 9. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a maior média de avaliação?

In [ ]:
(data[data['cuisines'] == 'Home-made'][['restaurant_name', 'restaurant_id', 'aggregate_rating']]
 .sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[False, True])).head(1)

,restaurant_name,restaurant_id,aggregate_rating
7525,Kanaat Lokantası,5914190,4.0


## 10. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a menor média de avaliação?


In [ ]:
(data[data['cuisines'] == 'Home-made'][['restaurant_name', 'restaurant_id', 'aggregate_rating']]
 .sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[True, True])).head(1)

,restaurant_name,restaurant_id,aggregate_rating
7373,GurMekan Restaurant,6007184,3.7


## 11. Qual o tipo de culinária que possui o maior valor médio de um prato para duas pessoas?

In [ ]:
(data[['cuisines', 'average_cost_for_two']]
.groupby('cuisines')
.mean()
.round(2)
.sort_values(by='average_cost_for_two', ascending=False)
.reset_index()).head(1)

,cuisines,average_cost_for_two
0,Modern Australian,1470693.06


## 12. Qual o tipo de culinária que possui a maior nota média?

In [ ]:
(data[['cuisines', 'aggregate_rating']]
.groupby('cuisines')
.max()
.round(2)
.sort_values(by='aggregate_rating', ascending=False)
.reset_index()).head(20)

,cuisines,aggregate_rating
0,Indian,4.9
1,Greek,4.9
2,Korean,4.9
3,Tapas,4.9
4,Japanese,4.9
5,Italian,4.9
6,International,4.9
7,Ice Cream,4.9
8,Tex-Mex,4.9
9,German,4.9


## 13. Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos online e fazem entregas?

In [ ]:
(data[['cuisines', 'is_delivering_now']].groupby('cuisines').sum().sort_values(by='is_delivering_now', ascending=False).reset_index()).head(1)

,cuisines,is_delivering_now
0,North Indian,317


# Visão Geral

### Métricas para:
- Número de restaurantes cadastrados
- Número de países cadastrados
- Número de cidades cadastradas
- Número de avaliaçoes feitas na plataforma
- Número de culinárias oferecidas

## 1. Número de restaurantes na base de dados:


In [ ]:
data['restaurant_id'].nunique()

6929

## 2. Número de países na base de dados


In [ ]:
data['country'].nunique()

15

## 3. Número de cidades na base de dados






In [ ]:
data['city'].nunique()

125

## 4. Total de avaliações feitas

In [ ]:
data['votes'].sum()

4194533

## 5. Número de culinárias oferecidas

In [ ]:
data['cuisines'].nunique()

165

# Visão Países

## 1. Quantidade de restaurantes por país


In [ ]:
df_aux = (data[['country', 'restaurant_name']]
.groupby('country')
.nunique()
.sort_values(by='restaurant_name', ascending=False)
.reset_index())

fig = px.bar(x=df_aux.country,
       y=df_aux.restaurant_name,
       title='Quantidade de restaurantes por país',
       labels={'x':'Países', 'y':'Número de restaurantes registrados'},
       color=df_aux.country)

fig.show()

## 2. Quantidade de cidades por país


In [ ]:
df_aux = (data[['country', 'city']]
.groupby('country')
.nunique()
.sort_values(by='city', ascending=False)
.reset_index())

fig = px.bar(x=df_aux.country,
       y=df_aux.city,
       title='Quantidade de cidades por país',
       labels={'x':'Países', 'y':'Número de cidades registradas'},
       color=df_aux.country)
fig.show()

## 3.  Média de avaliaçoes por restaurante em cada país







In [ ]:
np.arange(0, 5, 0.5)

array([0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5])

In [ ]:
df_aux = (data[['country', 'votes']]
.groupby('country')
.mean()
.round(2)
.sort_values(by='votes', ascending=False)
.reset_index())

fig = px.bar(x=df_aux.country,
       y=df_aux.votes,
       title='Quantidade média de avaliaçoes por país',
       labels={'x':'Países', 'y':'Quantidade média de avaliaçoes'},
       color=df_aux.country)

fig.show()

## 4. Média de notas por país

In [ ]:
df_aux = (data[['country', 'aggregate_rating']]
.groupby('country')
.mean()
.round(2)
.sort_values(by='aggregate_rating', ascending=False)
.reset_index())

fig = px.bar(x=df_aux.country,
       y=df_aux.aggregate_rating,
       title='Nota média dos restaurantes por país',
       labels={'x':'Países', 'y':'Nota média dos restaurantes'},
       color=df_aux.country)
fig = go.Figure(fig, layout_yaxis_range = [3, 5])
fig.show()

In [ ]:
(data[['country', 'votes']]
.groupby('country')
.sum()
.sort_values(by='votes', ascending=False)
.reset_index()).head(1)

,country,votes
0,India,2800164


# Visão Cidades

## 1. Dez cidades com mais restaurantes na base de dados


In [ ]:
df_aux = (data[['city', 'restaurant_id', 'country']]
.groupby(['city', 'country'])
.count()
.sort_values(by='restaurant_id', ascending=False)
.reset_index()).head(10)

fig = px.bar(x=df_aux.city,
             y=df_aux.restaurant_id,
             color=df_aux.country,
             labels={'x':'Cidades', 'y':'Número de restaurantes'},
             title='Dez cidades com mais restaurantes registrados')
fig.show()

## 2. Sete cidades com mais restaurantes com notas acima de 4


In [ ]:
df_aux = (data[data['aggregate_rating'] > 4][['city', 'country', 'restaurant_name']]
.groupby(['city', 'country'])
.nunique()
.sort_values(by='restaurant_name', ascending=False)
.reset_index()).head(7)

fig = px.bar(x=df_aux.city,
       y=df_aux.restaurant_name,
       color=df_aux.country,
       labels={'x':'Cidades', 'y':'Número de restaurantes'},
       title='Sete cidades com mais restaurantes com nota maior que 4,0')

fig.show()

## 3. Sete cidades com mais restaurantes com notas abaixo de 2.5







In [ ]:
df_aux = (data[data['aggregate_rating'] < 2.5][['city', 'country', 'restaurant_name']]
.groupby(['city', 'country'])
.nunique()
.sort_values(by='restaurant_name', ascending=False)
.reset_index()).head(7)

fig = px.bar(x=df_aux.city,
       y=df_aux.restaurant_name,
       color=df_aux.country,
       labels={'x':'Cidades', 'y':'Número de restaurantes'},
       title='Sete cidades com mais restaurantes com nota menor 2.5')

fig.show()

## 4. Dez cidades com mais tipos de restaurantes (culinárias) distintos

In [ ]:
df_aux = (data[['city', 'country', 'cuisines']]
.groupby(['city', 'country'])
.nunique()
.sort_values(by='cuisines', ascending=False)
.reset_index()).head(10)

fig = px.bar(x=df_aux.city,
       y=df_aux.cuisines,
       title='Dez cidade com mais culinárias distintas',
       labels={'x':'Cidades', 'y':'Número de culinárias distintas'},
       color=df_aux.country)

fig.show()

In [ ]:
(data[['country', 'votes']]
.groupby('country')
.sum()
.sort_values(by='votes', ascending=False)
.reset_index()).head(1)

,country,votes
0,India,2800164


# Visão Culinárias

### Métricas para:
Melhores Restaurantes dos Principais tipos Culinários
- Japanese
- Brazilian
- Italian
- Arabian
- Indian

## 1. Melhor restaurante japanese


In [ ]:
cols = ['aggregate_rating', 'restaurant_id', 'restaurant_name', 'average_cost_for_two', 'currency', 'votes', 'country', 'city']

In [ ]:
best_JP = (data[data.cuisines == 'Japanese'][cols]
           .sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[False, True])
           .reset_index(drop=True).head(1))
best_JP

,aggregate_rating,restaurant_id,restaurant_name,average_cost_for_two,currency,votes,country,city
0,4.9,6107336,Sushi Samba,110,Pounds(£),484,England,London


## 2. Melhor restaurante brazilian


In [ ]:
best_BR = data[data.cuisines == 'Brazilian'][cols].sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[False, True]).head(1).reset_index(drop=True)
best_BR

,aggregate_rating,restaurant_id,restaurant_name,average_cost_for_two,currency,votes,country,city
0,4.9,7300955,Braseiro da Gávea,100,Brazilian Real(R$),47,Brazil,Rio de Janeiro


## 3. Melhor restaurante italian







In [ ]:
best_IT = data[data.cuisines == 'Italian'][cols].sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[False, True]).head(1).reset_index(drop=True)
best_IT

,aggregate_rating,restaurant_id,restaurant_name,average_cost_for_two,currency,votes,country,city
0,4.9,10021,Darshan,700,Indian Rupees(Rs.),3106,India,Pune


## 4. Melhor restaurante arabian

In [ ]:
best_AB = data[data.cuisines == 'Arabian'][cols].sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[False, True]).head(1).reset_index(drop=True)
best_AB

,aggregate_rating,restaurant_id,restaurant_name,average_cost_for_two,currency,votes,country,city
0,4.7,18535007,Mandi@36,600,Indian Rupees(Rs.),1085,India,Hyderabad


## 5. Melhor restaurante indian


In [ ]:
best_IN = data[data.cuisines == 'Indian'][cols].sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[False, True]).head(1).reset_index(drop=True)
best_IN

,aggregate_rating,restaurant_id,restaurant_name,average_cost_for_two,currency,votes,country,city
0,4.9,207770,Mint Leaf of London,450,Emirati Diram(AED),2092,United Arab Emirates,Dubai


## 6. Dez cozinhas mais bem avaliadas


In [ ]:
# Gráfico de barras com culinária (eixo x) e nota média (eixo y)

df_aux = data.loc[data.cuisines != 'Others', :]
df_aux = (df_aux[['cuisines', 'aggregate_rating']]
.groupby('cuisines')
.mean()
.sort_values(by='aggregate_rating', ascending=False)
.reset_index()).head(10)


fig = px.bar(x=df_aux.cuisines,
             y=df_aux.aggregate_rating,
             title="Dez tipos de culnária melhor avaliados",
             labels={'x': 'Tipos de culinária', 'y':'Média de notas'})
fig = go.Figure(fig, layout_yaxis_range = [4.5, 5])
fig.show()

## 7. Dez cozinhas mais mal avaliadas


In [ ]:
df_aux = data.loc[data.cuisines != 'Others', :]
df_aux = (df_aux[['cuisines', 'aggregate_rating']]
.groupby('cuisines')
.mean()
.sort_values(by='aggregate_rating', ascending=True)
.reset_index()).head(10)

fig = px.bar(x=df_aux.cuisines, y=df_aux.aggregate_rating, title="Dez tipos de culinária pior avaliados", labels={'x': 'Tipos de culinária', 'y':'Média de notas'})
fig = go.Figure(fig, layout_yaxis_range = [0, 5])
fig.show()

## 8. Quantidade de restaurante por tipo de de cozinha


In [ ]:
data.cuisines.nunique()

165

In [ ]:
df_aux = (data[['cuisines', 'restaurant_id']]
          .groupby('cuisines')
          .count()
          .sort_values(by='restaurant_id', ascending=False)
          .reset_index()).head(10)

fig = px.bar(x=df_aux.cuisines,
              y=df_aux.restaurant_id,
              title="Dez tipos de culinária com maior número de restaurantes",
              labels={'x': '', 'y':'Quantidade de restaurantes'})
# fig = go.Figure(fig, layout_yaxis_range = [])
# fig.update_xaxes(tickangle=45)
fig.show()

## 9. Parcela de Mercado das Dez Culinárias Mais Comuns

In [ ]:
print(px.colors.qualitative.Safe) #Pegando a paleta de cores a serem utilizadas no gráfico

['rgb(136, 204, 238)', 'rgb(204, 102, 119)', 'rgb(221, 204, 119)', 'rgb(17, 119, 51)', 'rgb(51, 34, 136)', 'rgb(170, 68, 153)', 'rgb(68, 170, 153)', 'rgb(153, 153, 51)', 'rgb(136, 34, 85)', 'rgb(102, 17, 0)', 'rgb(136, 136, 136)']


In [ ]:
df_aux = (data[['cuisines', 'restaurant_id']]
          .groupby('cuisines')
          .count()
          .sort_values(by='restaurant_id', ascending=False)
          .reset_index()).head(10)
fig = px.pie(df_aux,
             values='restaurant_id',
             names='cuisines',
             title="Parcela de mercado das dez culinárias mais comuns",
             color_discrete_sequence=['rgb(136, 204, 238)',
             'rgb(204, 102, 119)',
             'rgb(221, 204, 119)',
             'rgb(17, 119, 51)',
             'rgb(51, 34, 136)',
             'rgb(170, 68, 153)',
             'rgb(68, 170, 153)',
             'rgb(153, 153, 51)',
             'rgb(136, 34, 85)',
             'rgb(102, 17, 0)',
             'rgb(136, 136, 136)'])

fig.show()

In [ ]:
df_aux = data.loc[data.cuisines != 'Others', :]
df_aux = (df_aux[['cuisines', 'has_online_delivery', 'aggregate_rating']]
          .groupby('cuisines')
          .agg({'has_online_delivery': 'sum', 'aggregate_rating':'mean'})
          .round(2)
          .sort_values(by='has_online_delivery', ascending=False)
          .reset_index()).head(10)
df_aux

fig = px.sunburst(df_aux,
                  path = ['cuisines', 'has_online_delivery'],
                  values = 'has_online_delivery',
           color = 'aggregate_rating',
           color_continuous_scale = 'PuBu',
           color_continuous_midpoint = np.average(df_aux['aggregate_rating']))
fig.update_layout(coloraxis_colorbar_title='Nota média')
fig.show()

In [ ]:
def a(b):
  c = b*b
  return c

In [ ]:
b = 5
c = 7

print(a(c))

49
